# Statistical arbitrage
Cross sectional analysis sketch for identifying cointegrated pairs

### Load config

In [1]:
import yaml
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import *

with open("../config.yaml") as f:
    config = yaml.safe_load(f)

trading_client = TradingClient(config['alpaca']['key'], config['alpaca']['secret'])
stock_client = StockHistoricalDataClient(config['alpaca']['key'], config['alpaca']['secret'])

### Get asset universe and historical data

Example:
* 100 symbols
* Daily prices for 1 year

In [2]:
assets = list(map(dict, trading_client.get_all_assets(GetAssetsRequest(status=AssetStatus.ACTIVE))))
assets = pd.DataFrame(assets).set_index("id")

In [3]:
asset_filter = assets.tradable & assets.marginable & assets.shortable & assets.fractionable

assets[asset_filter].drop([
    "status",
    "tradable",
    "marginable",
    "shortable",
    "easy_to_borrow",
    "fractionable",
    "min_order_size",
    "min_trade_increment",
    "price_increment"
], axis=1).to_csv("assets.csv")

In [4]:
symbol_list = list(assets[asset_filter].symbol)[:100]

In [5]:
request_params = StockBarsRequest(
                        symbol_or_symbols=symbol_list,
                        timeframe=TimeFrame.Day,
                        start=datetime.datetime(2022, 1, 1),
                        end=datetime.datetime(2023, 1, 1)
                 )

bars = stock_client.get_stock_bars(request_params).df

Log returns sample

In [6]:
close = bars.close.unstack().transpose().dropna()
log_returns = np.log(close.pct_change() + 1).dropna()
corr_matrix = log_returns.corr()
returns_correlation = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)).stack().sort_values(ascending=False)

returns_correlation

symbol  symbol
DGL     SGOL      0.992079
DGRO    DGRW      0.984106
DFAI    DFAX      0.981418
DGRO    DIA       0.980256
DBP     DGL       0.976223
                    ...   
DGRW    SH       -0.957802
MOAT    SH       -0.957904
DGRO    SH       -0.961885
DIA     SH       -0.963928
DFAC    SH       -0.987957
Length: 4950, dtype: float64

### Pairwise price cointegration p-values

Pairwise calculation of 251 data points (1 year EOD prices) for 100 assets (4950 pairs) took ~1m 43.5s.

In [15]:
close.shape

(251, 100)

In [7]:
from statsmodels.tsa.api import coint
import itertools

coint_result = pd.DataFrame(columns=close.keys(), index=close.keys())

for pair in itertools.combinations(close.keys(), 2):
    # p values
    coint_result.loc[pair] = coint(close[pair[0]], close[pair[1]])[1]


In [8]:
coint_pvalues = coint_result.unstack().sort_values().dropna()

At the 1% significance level there were 295 pairs (>5% of the 4950 pairs) identified (fluke?).

In [9]:
coint_pvalues[(coint_pvalues < 0.01) & (coint_pvalues > 0)]

symbol  symbol
RCKY    DDL            0.0
DGNU    DDL            0.0
DEN     DDL            0.0
MNMD    DDL            0.0
RHI     DDL            0.0
                    ...   
MNSO    MNRO      0.009905
RCL     RBLX      0.009951
MODG    DIN       0.009969
SGRY    MNDY       0.00999
MNST    DKNG      0.009991
Length: 295, dtype: object

Note rounding error in display

In [10]:
coint_pvalues[("RCKY", "DDL")]

3.217502448206104e-08